In [0]:
%matplotlib inline
import glob
import matplotlib
from matplotlib import pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import imageio as im

from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import load_img
from keras.callbacks import ModelCheckpoint


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
images = []
for img_path in glob.glob('/content/gdrive/My Drive/training_set/squares/*.png'):
  images.append(mpimg.imread(img_path))

In [0]:
plt.figure(figsize = (20, 10))
columns = 5
for i, image in enumerate(images):
  plt.subplot(len(images) / columns + 1, columns, i + 1)
  plt.imshow(image)

In [0]:
images = []
for img_path in glob.glob('/content/gdrive/My Drive/training_set/circles/*.png'):
  images.append(mpimg.imread(img_path))

In [0]:
plt.figure(figsize = (20, 10))
columns = 5
for i, image in enumerate(images):
  plt.subplot(len(images) / columns + 1, columns, i + 1)
  plt.imshow(image)

In [0]:
images = []
for img_path in glob.glob('/content/gdrive/My Drive/training_set/triangles/*.png'):
  images.append(mpimg.imread(img_path))

In [0]:
plt.figure(figsize = (20, 10))
columns = 5
for i, image in enumerate(images):
  plt.subplot(len(images) / columns + 1, columns, i + 1)
  plt.imshow(image)

In [0]:
images = []
for img_path in glob.glob('/content/gdrive/My Drive/training_set/stars/*.PNG'):
  images.append(mpimg.imread(img_path))

In [0]:
plt.figure(figsize = (20, 10))
columns = 5
for i, image in enumerate(images):
  plt.subplot(len(images) / columns + 1, columns, i + 1)
  plt.imshow(image)

* 3 -> rgb, 1 = 흑백, 3 = 컬러, 4 = 컬러 + hue(흐릿한 정도) 

In [0]:
img = im.imread('/content/gdrive/My Drive/training_set/circles/drawing(35).png')
img.shape


In [0]:
img = im.imread('/content/gdrive/My Drive/training_set/stars/91.PNG')
img.shape

In [0]:
cf = Sequential() #cf = classifier

In [0]:
cf.add(Conv2D(32, (3, 3), padding = 'same', input_shape = (28, 28, 3), activation = 'relu'))
cf.add(Conv2D(32, (3, 3), activation = 'relu'))
cf.add(MaxPooling2D(pool_size = (2, 2)))
cf.add(Dropout(0.5)) #0.5 = 절반 제거 #antes era 0.25

In [0]:
cf.add(Conv2D(64, (3, 3), padding = 'same', activation = 'relu'))
cf.add(Conv2D(64, (3, 3), activation = 'relu'))
cf.add(MaxPooling2D(pool_size = (2, 2)))
cf.add(Dropout(0.5))

In [0]:
cf.add(Conv2D(64, (3, 3), padding = 'same', activation = 'relu'))
cf.add(Conv2D(64, (3, 3), activation = 'relu'))
cf.add(MaxPooling2D(pool_size = (2, 2)))
cf.add(Dropout(0.5))

In [0]:
cf.add(Flatten())

In [0]:
cf.add(Dense(units = 512, activation = 'relu'))
cf.add(Dropout(0.5))
cf.add(Dense(units = 4, activation = 'softmax'))

In [0]:
cf.summary()

In [0]:
#compiling the CNN
cf.compile(optimizer = 'rmsprop',
           loss = 'categorical_crossentropy',
            metrics = ['accuracy'])

In [0]:
train_datagen = ImageDataGenerator(rescale = 1./255)
test_datagen = ImageDataGenerator(rescale = 1./255)
training_set = train_datagen.flow_from_directory('/content/gdrive/My Drive/training_set',
                                                target_size = (28, 28),
                                                batch_size = 16, #클수록 좋음
                                                class_mode = 'categorical')

In [0]:
test_set = test_datagen.flow_from_directory('/content/gdrive/My Drive/test_set',
                                            target_size = (28, 28),
                                            batch_size = 16,
                                            class_mode = 'categorical')

In [0]:
checkpointer = ModelCheckpoint(filepath = "best_weights.hdf5",
                                monitor = 'val_acc',
                                verbose = 1,
                                save_best_only = True)

In [0]:
from keras.utils import plot_model
plot_model(cf, to_file = './model.png', show_shapes = True)
plot_model(cf, to_file='./model.svg', show_shapes = True)

In [0]:
from IPython.display import Image
Image('./model.png')

In [0]:
history = cf.fit_generator(training_set,
                            steps_per_epoch = 100,
                            epochs = 40,
                            callbacks = [checkpointer],
                            validation_data = test_set,
                            validation_steps = 50)

In [0]:
cf.load_weights('best_weights.hdf5')

In [0]:
cf.save('shapes_cnn.h5')

In [0]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

In [0]:
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label = 'Training acc')
plt.plot(epochs, val_acc, 'b', label = 'Validation acc')
plt.title('Training amd validation accuracy')
plt.legend()

In [0]:
plt.figure()

In [0]:
plt.plot(epochs, loss, 'bo', label = 'Training loss')
plt.plot(epochs, val_loss, 'b', label = 'Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [0]:
img_path = '/content/gdrive/My Drive/test_set/circles/drawing(2).png'

img = image.load_img(img_path, target_size = (28, 28))
img_tensor = image.img_to_array(img)
img_tensor = np.expand_dims(img_tensor, axis = 0)
img_tensor /= 255.
plt.imshow(img_tensor[0])
plt.show()
print(img_tensor.shape)

In [0]:
x = image.img_to_array(img)
x = np.expand_dims(x, axis = 0)

images = np.vstack([x])
classes = cf.predict_classes(images, batch_size = 10)
print("Predicted class is : ", classes)

In [0]:
img_path = '/content/gdrive/My Drive/test_set/stars/19.PNG'

img = image.load_img(img_path, target_size = (28, 28))
img_tensor = image.img_to_array(img)
img_tensor = np.expand_dims(img_tensor, axis = 0)
img_tensor /= 255.
plt.imshow(img_tensor[0])
plt.show()
print(img_tensor.shape)

In [0]:
x = image.img_to_array(img)
x = np.expand_dims(x, axis = 0)

images = np.vstack([x])
classes = cf.predict_classes(images, batch_size = 10)
print("Predicted class is : ", classes)

In [0]:
layer_outputs = [layer.output for layer in cf.layers[:12]]
activation_model = models.Model(inputs = cf.intput, outputs = layer_outputs)